A simple fix for untitled-3's problem, accuracy same as guessing, the model is really not working.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

data_buf = np.load('data.npy')
index_buf = np.load('index.npy')
data = data_buf[:1055000]
index = index_buf[index_buf < 1055000].astype(int) 
print(data.shape)
print(index)

(64, 6122790)
[  25351   32967   65124  110605  110614  139796  139803  179850  179858
  220048  231137  245534  270780  270789  309584  429010  477457  483625
  483633  520625  525407  525415  536316  536324  565340  689253  713584
  720463  725368  807271  852387  923113  972713  985866 1009480 1046297
 1052564]


Sliding Window: try 20ms length (600 samples) and 5ms step (150 samples) for each window?

In [3]:
# define a sliding window
def sliding_window(data, window_size=600, stride=150):
    window_data = np.zeros(((len(data)-window_size)//stride, window_size))
    index = np.zeros(((len(data)-window_size)//stride, window_size))
    for row in range((len(data)-window_size)//stride):
        window_data[row, :] = data[row*stride:row*stride+window_size]
        index[row, :] = np.arange(row*stride, row*stride+window_size)
    return window_data, index

windowed_data, windowed_index = sliding_window(data[53, :], window_size=600, stride=150)

window_data_low, window_index_low = sliding_window(data[53, :850000], window_size=600, stride=150)
window_data_high, window_index_high = sliding_window(data[53, 850000:1055000], window_size=600, stride=150)

label = np.zeros(windowed_data.shape[0])
for i in range(windowed_data.shape[0]):
    if np.any(np.isin(windowed_index[i, :], index)):
        label[i] = 1
    else:
        label[i] = 0
        
        
def make_label(windowed_data, index):
    label = np.zeros(windowed_data.shape[0])
    for i in range(windowed_data.shape[0]):
        if np.any(np.isin(windowed_index[i, :], index)):
            label[i] = 1
        else:
            label[i] = 0
    return label

label_low = make_label(window_data_low, index)
label_high = make_label(window_data_high, index)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=7, padding='same')
        self.bn1 = nn.BatchNorm1d(num_features=16)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding='same')
        self.bn2 = nn.BatchNorm1d(num_features=32)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding='same')
        self.bn3 = nn.BatchNorm1d(num_features=64)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        # fc
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

def balance_data(data_tensor, labels):
    positive_index = np.where(labels == 1)[0]
    negative_index = np.where(labels == 0)[0]
    negative_index = np.random.choice(negative_index, size=positive_index.shape[0], replace=False)
    balanced_data_tensor = torch.cat((data_tensor[positive_index], data_tensor[negative_index]), dim=0)
    balanced_label_tensor = torch.cat((labels[positive_index], labels[negative_index]), dim=0)
    return balanced_data_tensor, balanced_label_tensor

In [5]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

data_low_tensor = torch.tensor(window_data_low[:, None, :], dtype=torch.float32)
data_high_tensor = torch.tensor(window_data_high[:, None, :], dtype=torch.float32)
label_low_tensor = torch.tensor(label_low, dtype=torch.float32).unsqueeze(1)
label_high_tensor = torch.tensor(label_high, dtype=torch.float32).unsqueeze(1)

# data_tensor = torch.tensor(windowed_data[:, None, :], dtype=torch.float32)
# label_tensor = torch.tensor(label, dtype=torch.float32).unsqueeze(1)

X_train, y_train = balance_data(data_low_tensor, label_low_tensor)
X_test, y_test = balance_data(data_high_tensor, label_high_tensor)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# X_train, X_test, y_train, y_test = train_test_split(data_tensor, label_tensor, test_size=0.2, stratify=label_tensor)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# initial model
model = ConvNet()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 40

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 5 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

model.eval()
all_predicted = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs.data > 0.5).float()
        all_predicted.extend(predicted.view(-1).tolist())
        all_labels.extend(labels.view(-1).tolist())

accuracy = accuracy_score(all_labels, all_predicted)
conf_matrix = confusion_matrix(all_labels, all_predicted)
precision = precision_score(all_labels, all_predicted)
recall = recall_score(all_labels, all_predicted)
f1 = f1_score(all_labels, all_predicted)

print(f'Accuracy on test set: {accuracy * 100:.2f}%')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

torch.Size([186, 1, 600]) torch.Size([50, 1, 600]) torch.Size([186, 1]) torch.Size([50, 1])
Epoch 5/40, Loss: 0.016855869442224503
Epoch 10/40, Loss: 0.003011260647326708
Epoch 15/40, Loss: 0.004418177530169487
Epoch 20/40, Loss: 0.005040186457335949
Epoch 25/40, Loss: 0.0021974400151520967
Epoch 30/40, Loss: 0.000330679293256253
Epoch 35/40, Loss: 0.0011856112396344543
Epoch 40/40, Loss: 0.00022112971055321395
Accuracy on test set: 48.00%
Confusion Matrix:
[[24  1]
 [25  0]]
Precision: 0.00
Recall: 0.00
F1 Score: 0.00


In [6]:
# import matplotlib.pyplot as plt

# plt.figure()
# for i in range(len(index)):
#     plt.plot(np.arange(0, 600), data[53, index[i]-200:index[i]+400])
# plt.show()